In [17]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model


#Load the pickle file

with open ("Label_Encoder_Gender.pkl","rb") as file:
    Label_Encoder_Gender = pickle.load(file)
    
with open ("OHE_Geographhy.pkl","rb") as file:
    OHE_Geographhy =pickle.load(file)
    
with open ("Standard_Scaler.pkl","rb") as file:
    Standard_Scaler = pickle.load(file)

model = load_model("ANN_model.h5")

In [39]:
prediction_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 30,
    "Balance":60000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}

In [44]:
df = pd.DataFrame(prediction_data,index=['CreditScore'])

In [49]:
df=df.reset_index(drop=True)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,30,60000,2,1,1,50000


In [50]:
#One hot encoding of Geography

df_Geography = OHE_Geographhy.transform(df[["Geography"]])
df_Geography

<1x3 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [51]:
df_Geography.toarray()

array([[1., 0., 0.]])

In [52]:
OHE_Geographhy.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [53]:
df_geo_final = pd.DataFrame(data=df_Geography.toarray(),columns=OHE_Geographhy.get_feature_names_out())
df_geo_final

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [54]:
#Label Encoding of Gender
df["Gender"]=Label_Encoder_Gender.transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,30,60000,2,1,1,50000


In [55]:
#Concatinating

df_Final = pd.concat([df.drop(labels=["Geography"],axis=1),df_geo_final],axis=1)
df_Final

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,30,60000,2,1,1,50000,1.0,0.0,0.0


In [56]:
df_Final_Scaled = Standard_Scaler.transform(df_Final)


In [58]:
prediction = model.predict(df_Final)
prediction

1/1 [==============================] - 0s 40ms/step


array([[1.]], dtype=float32)

In [59]:
predict_proba = prediction[0]
predict_proba

array([1.], dtype=float32)

In [60]:
predict_proba = prediction[0][0]
predict_proba

1.0

In [61]:
if predict_proba > 0.5:
    print("Churned")
else:
    print("No Churned")

Churned
